In [14]:
from difflib import SequenceMatcher
import pandas as pd
import numpy as np
import re
import pickle
from datetime import datetime
from pprint import pprint
from __future__ import division
import matplotlib.pyplot as plt 
import matplotlib
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from patsy import dmatrices
from patsy import dmatrix
from collections import defaultdict
from sklearn.linear_model import Lasso
from sklearn import cross_validation
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.special import factorial
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn import grid_search, datasets
from sklearn import linear_model
from sklearn.learning_curve import learning_curve
import math
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import roc_curve, auc
from sklearn import tree
from sklearn import svm, grid_search
from sklearn.externals.six import StringIO
from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime
import sklearn as sk
import pydot 
import os
import qgrid
from matplotlib.colors import ListedColormap
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.lda import LDA
from sklearn.qda import QDA
import requests
from bs4 import BeautifulSoup
import nltk
from textblob import TextBlob
from nltk.util import ngrams
from collections import defaultdict
from operator import itemgetter
from nltk.corpus import stopwords
from textblob.sentiments import NaiveBayesAnalyzer
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
import urllib2
%matplotlib inline 

In [200]:
with open('id_dict.pkl', 'r') as picklefile:
     id_dict = pickle.load(picklefile)
with open('cat_dict.pkl', 'r') as picklefile:
     cat_dict = pickle.load(picklefile)
with open('ex_df.pkl', 'r') as picklefile:
     ex = pickle.load(picklefile)
with open('ex2_df.pkl', 'r') as picklefile:
     ex2 = pickle.load(picklefile)
with open('ex3.pkl', 'r') as picklefile:
    ex3 = pickle.load(picklefile)
with open('pop_ids.pkl', 'r') as picklefile:
     pop_ids= pickle.load(picklefile)
with open('url_dict.pkl', 'r') as picklefile:
     url_dict = pickle.load(picklefile)
with open('test_urls.pkl', 'r') as picklefile:
     test_urls = pickle.load(picklefile)
with open('mini_dfs.pkl', 'r') as picklefile:
     minis = pickle.load(picklefile)

In [34]:
with open('att_dict.pkl', 'r') as picklefile:
     atts = pickle.load(picklefile)

In [83]:
test_urls

[u'Hiro-Sushi-Scottsdale',
 u'Zinc-Bistro-Scottsdale',
 u'Oreganos-Pizza-Bistro-Scottsdale',
 u'Houstons-Restaurant-Scottsdale',
 u'Lox,-Stock-&-Bagel-Scottsdale',
 u'RA-Sushi-Bar-Restaurant-Scottsdale',
 u'Ristorante-Giuseppe-Scottsdale',
 u'Flos-Scottsdale',
 u'The-Melting-Pot-Scottsdale',
 u'The-Cheesecake-Factory-Scottsdale',
 u'Havana-Cafe-Scottsdale',
 u'Casellas-Italian-Delicatessen-Scottsdale',
 u'Randys-Restaurant-&-Ice-Cream-Scottsdale',
 u'Arrivederci-Scottsdale',
 u'Tutti-Santi-By-Nina-Scottsdale',
 u'The-Good-Egg---Hilton-Village-Scottsdale',
 u'Rays-Pizza-Scottsdale',
 u'Benihana-Scottsdale',
 u'Los-Olivos-Norte-Scottsdale']

In [97]:
test_url = 'http://www.yelp.com/biz/hiro-sushi-scottsdale'
site = requests.get(test_url)
page = site.text
soup = BeautifulSoup(page)

In [105]:
explore = soup.find(class_= 'menu-explore')
explore

<a class="menu-explore" href="/menu/hiro-sushi-scottsdale">Full menu</a>

In [109]:
explore['href']

'/menu/hiro-sushi-scottsdale'

In [117]:
test_url = 'http://www.yelp.com/biz/hiro-sushi-scottsdale'+explore['href']
site = requests.get(test_url)
page = site.text
soup = BeautifulSoup(page)

In [119]:
soup.find(class_='menu-sections')

In [63]:
def get_restos(dataset, start=700001, num_dfs=15, top_x=10):
    df_dict = {}
    for i in range(start, (start + num_dfs*10000), 10000):
        df = dataset[i]
        df['name'] = df['business_id'].apply(lambda x: id_dict[x])
        df['cats'] = df['name'].apply(lambda x: cat_dict[x])
        df['CityName'] = df['business_id'].apply(lambda x: url_dict[x])
        df['restos'] = df['cats'].apply(lambda x: 1 if 'Restaurants' in x else 0)
        df = df[['name', 'cats', 'stars', 'text', 'date', 'user_id', 'restos', 'CityName']]
        url_list = df.pivot_table('stars', index='CityName', columns='restos', aggfunc='count', margins=True).sort(1, ascending=False)
        urls = url_list.reset_index()['CityName'].tolist()[1:top_x+1]
        df_dict[i] = urls, df
    return df_dict

In [99]:
def screlpy(cityname):
    url = get_url(cityname)
    if url:
        site = requests.get(url)
        page = site.text
        soup = BeautifulSoup(page)
        explore = soup.find(class_= 'menu-explore')
        if explore:
            url = 'http://www.yelp.com' + explore['href']
            site = requests.get(url)
            page = site.text
            soup = BeautifulSoup(page)
            main = soup.find(class_='menu-sections')
            menu = {}
            course_list = []
            options = []
            for items in main:
                courses = main.find_all(class_="menu-section-header")
                for course in courses:
                    course_list.append(course.find('h2').text.strip())
                sections = soup.find_all(class_ = 'menu-section')
                for section in sections:
                    food_list = []
                    foods = section.find_all('h3')
                    for food in foods:
                        food_list.append(food.text.strip())
                    options.append(food_list)

            menu = dict(zip(course_list, options))
        else:
            menu = 'Menu not on Yelp'
            print cityname + ' Menu not on Yelp'
    else:
        menu = 'Page Not Found'
    return menu

In [212]:
def get_url(cityname, tries=5):
    cityname = cityname.replace(u'\xe9', 'e')
    cityname = cityname.replace(u'\xea', 'e')
    cityname = cityname.replace('&', 'and')
    cityname = cityname.replace('+', 'and')
    cityname = cityname.replace('---', '-')
    i = 0
    url = 'http://www.yelp.com/biz/'+ str(cityname)
    r = requests.get(url)
    try:
        url = 'http://www.yelp.com/biz/'+ str(cityname)
        r = requests.get(url)
    except:
        if (r.status_code != 200) & (i < tries+1):
            i += 1
            cityname = cityname + '-' + str(i)
        else:
            url = ''
    return url

In [46]:
check_url = test_urls[0]

In [47]:
menu = screlpy(check_url)
pprint(menu)

{u'Appetizers': [u'Edamame',
                 u'Hiyayakko',
                 u'Shumai',
                 u'Yakitori',
                 u'Vegetatable Tempura'],
 u'Desserts': [u'Ice Cream', u'Tempura Ice Cream'],
 u'Dinner': [u'Chicken Teriyaki',
             u'Sanma Shioyaki',
             u'Saba Shiyoki',
             u'Ginger Pork',
             u'Salmon Teriyaki',
             u'Mixed Tempura',
             u'Fried Oyasters',
             u'Chicken Or Pork Katsu',
             u'Shrimp Tempura'],
 u'Noodles': [u'Zaru Soba',
              u'Chicken Udon',
              u'Tempura Udon',
              u'Okinara Soba',
              u'Tenzaru'],
 u'Salads': [u'House Salad', u'Sunomono', u'Gomaae', u'Takosu']}


In [ ]:
for url in test_urls:
    menu = screlpy(url)
    pprint(menu)

In [ ]:
def get_menus(NameCities, menus=None):
    if not menus:
        menus = {}
    else:
        menus = menus
    for NameCity in NameCities:
        menus[NameCity] = screlpy(NameCity)
        print 'screlped ' + str(NameCity)
    return menus

In [65]:
def word_match(words, menu, vector_dict, threshold=70):
    matches = []
    for word in words:
        for key, value in menu.iteritems():
            for item in value:
                score = fuzz.ratio(word, item)
                if score > threshold:
                    matches.append((word, item, vector_dict[word]))
    if len(matches) < 5:
        for word in words:
            for key, value in menu.iteritems():
                for item in value:
                    short_item = ' '.join(item.split()[-2:])
                    score = fuzz.ratio(word, short_item)
                    if (score > threshold):
                        matches.append((word, item, vector_dict[word]))

    return matches

In [54]:
def best_matches(ngrams, menu, vector_dict, threshold=65):
    best_matches = {}
    for ngram in ngrams:
        biggest = 0
        for key, value in menu.iteritems():
            for item in value:
                temp = {}
                score = fuzz.token_set_ratio(ngram, item)
                if score > threshold:
                    score2 = fuzz.partial_ratio(ngram, item)
                    score3 = fuzz.ratio(ngram, item)
                    temp[item] = score2+score3
                    if temp[item] > biggest:
                        biggest = temp[item]
                        best_matches[ngram] = item, vector_dict[ngram]
    if len(best_matches) < 5:
        for ngram in ngrams:
            biggest = 0
            for key, value in menu.iteritems():
                for item in value:
                    temp = {}
                    short_item = ' '.join(item.split()[-2:])
                    score = fuzz.token_set_ratio(ngram, short_item)
                    if score > threshold:
                        score2 = fuzz.partial_ratio(ngram, short_item)
                        score3 = fuzz.ratio(ngram, short_item)
                        temp[item] = score2+score3
                        if temp[item] > biggest:
                            biggest = temp[item]
                            best_matches[ngram] = item, vector_dict[ngram]
        
    return best_matches

In [55]:
def get_keyword_counts(column):
    text = column
    vectorizer = CountVectorizer(ngram_range=(1,3), stop_words='english')
    vectorizer.fit(text)
    features = vectorizer.get_feature_names()
    x = vectorizer.transform(column).toarray()
    counts = pd.DataFrame(x, columns=features)
    singles = counts.applymap(lambda x: 1 if x > 0 else 0)
    return singles

In [ ]:
check_column = ex2.text
counts = get_keyword_counts(check_column)
counts

In [ ]:
sums = []
for (index, value) in counts.sum().iteritems():
    sums.append((value, index))

In [ ]:
sorted(sums, reverse=True)

In [56]:
def get_weights(df, column=ex2['stars']):
    singles = df.copy().reset_index() 
    weighted = singles.join(column.reset_index(), lsuffix = '_0')
    totals = weighted.append(weighted.sum(numeric_only=True), ignore_index=True)
    dfs = {}
    for i in range(1,6):
        dfs[i] = weighted[weighted['stars']== i]
    return dfs

In [ ]:
dfs = get_weights(counts, ex2['stars'])

In [57]:
def get_totals(df_list):
    totals = {}
    for i in range(1,6):
        counts = {}
        df = df_list[i]
        for (index, value) in df.sum().iteritems():
            counts[index] = value
        totals[i] = counts
    return totals

In [ ]:
totals = get_totals(dfs)

In [58]:
def get_vectors(totals):
    vectors = {}
    for key in totals[1].keys():
        vectors[key] = sum([totals[1][key],totals[2][key],totals[3][key],totals[4][key],totals[5][key]]), totals[1][key],totals[2][key],totals[3][key],totals[4][key],totals[5][key]
    return vectors

In [ ]:
vectors = get_vectors(totals)
vectors.keys()

In [ ]:
popular = []
for key, value in vectors.iteritems():
    popular.append((value, key))

pprint(sorted(popular, reverse=True))

In [ ]:
def filter_keys(vector_dict):
    bigkeys = filter(lambda x: len(x.split()) > 1, vector_dict.keys())
    bigkeys = filter(lambda x: vector_dict[x][0] > 5, bigkeys)
    
    littlekeys = filter(lambda x: len(x.split()) == 1, vector_dict.keys())
    littlekeys = filter(lambda x: vector_dict[x][0] > 7, littlekeys)
    
    return littlekeys, bigkeys

In [97]:
def get_recs(vector_dict, menu):
    recs = []
    matches1 = word_match(filter_keys(vector_dict)[0], menu, vector_dict)
    matches2 = best_matches(filter_keys(vector_dict)[1], menu, vector_dict)
    for match in matches1:
        recs.append((get_scores(match[2]), match[0], match[1]))
    for key, value in matches2.iteritems():
        recs.append((get_scores(value[1]), key, value[0]))
    rdf = pd.DataFrame(recs)
    rdf['Score'] = rdf[0].apply(lambda x: x[1])
    rdf['Trendiness'] = rdf[0].apply(lambda x: x[0])
    rdf['Controversy'] = rdf[0].apply(lambda x: x[3])
    rdf['Buzz'] = rdf[0].apply(lambda x: x[2])
    rdf.rename(columns={1: 'Keyword', 2: 'MenuItem'}, inplace=True)
    rdf.drop(0, axis=1, inplace=True)
    rdf['Score'] = rdf['Score'].apply(lambda x: (x+2)*2.5)
    p_norm = rdf['Controversy'].max()
    b_norm = rdf['Trendiness'].max()
    rdf['Controversy'] = rdf['Controversy'].apply(lambda x: 10*x/p_norm)
    rdf['Trendiness'] = rdf['Trendiness'].apply(lambda x: 10*x/b_norm)
    return rdf.drop_duplicates()

In [215]:
def get_scores(vector):
    score = (2*vector[5] + vector[4] - vector[2] - 2*vector[1]) / vector[0]
    unweighted = (2*vector[5] + vector[4] - vector[2] - 2*vector[1])
    polarity = np.var(vector[5]*[5] + vector[4]*[4] + vector[3]*[3] + vector[2]*[2] + vector[1]*[1])
    buzz = vector[0]
    
    return unweighted, score, buzz, polarity

In [ ]:
rdf = get_recs(vectors, menu)
rdf

In [61]:
def final_recs(df):
    return df.groupby('MenuItem').agg({'Score': np.mean,'Trendiness': np.mean, 'Controversy': np.mean, 'Buzz': np.sum}).sort(['Trendiness', 'Score'], ascending=False)

In [ ]:
final_recs(rdf)

In [ ]:
len(ex3)

In [142]:
def recs_collector(db_dict, rec_dict=None, menu_list=None, start=700001, num_dfs=2):
    if not rec_dict:
        rec_dict = {}
    if not menu_list:
        menu_list = []
    for i in range(start, (start + num_dfs*10000), 10000):
        db = db_dict[i]
        menus = get_menus(db[0])
        ex = db[1].copy()
        for key, menu in menus.iteritems():
            key = key.replace(u'\xe9', 'e')
            key = key.replace(u'\xea', 'e')
            if key not in rec_dict:
                if menu == 'Menu not on Yelp':
                    rec_dict[key] = 'Menu not on Yelp'
                    print str(key) + "'s Menu not on Yelp =("
                else:
                    if key in ex['CityName'].unique():
                        df = ex[ex['CityName']==key]
                        counts = get_keyword_counts(df.text)
                        dfs = get_weights(counts, df['stars'])
                        totals = get_totals(dfs)
                        vectors = get_vectors(totals)
                        rdf = get_recs(vectors, menu)
                        final = final_recs(rdf)
                        rec_dict[key] = [final, rdf]
                        print 'Rec Made on ' + str(key)
                    else:
                        rec_dict[key] = 'No reviews found'
                        print "Couldn't find reviews on " + str(key)
            if menu not in menu_list: 
                menu_list.append((key, menu))
    return menu_list, rec_dict

In [ ]:
db_dict = get_restos(minis)

In [151]:
menus, recs_db = recs_collector(db_dict, rec_dict=recs_db, menu_list=menus, start=700001)

screlped!
screlped!
Cartel-Coffee-Lab-Tempe Menu not on Yelp
screlped!
Morels-French-Steakhouse-&-Bistro-Las-Vegas Menu not on Yelp
screlped!
Kabuki-Japanese-Restaurant-Tempe Menu not on Yelp
screlped!
Bad-Daddys-Burger-Bar---Dilworth-Charlotte Menu not on Yelp
screlped!
screlped!
Forefathers-Gourmet-Cheesesteaks-&-Fries-Tempe Menu not on Yelp
screlped!
screlped!
screlped!
screlped!
screlped!
screlped!
The-Breadfruit-&-Rum-Bar-Phoenix Menu not on Yelp
screlped!
screlped!
screlped!
screlped!
Espressamente-Illy-Las-Vegas Menu not on Yelp
screlped!
screlped!
screlped!


In [181]:
menus3, recs_db3 = recs_collector(db_dict, rec_dict=recs_db3, menu_list=menus3, start=800001,  num_dfs=4)

screlped!
Au-Pied-de-Cochon-Montréal Menu not on Yelp
screlped!
screlped!
screlped!
Blue-Fin-Sushi-&-Roll-Las-Vegas Menu not on Yelp
screlped!
screlped!
screlped!
screlped!
screlped!
screlped!
Rec Made on Montesanos-Eateria-Las-Vegas
Rec Made on LAVO-Italian-Restaurant-&-Lounge-Las-Vegas
Rec Made on Cafe-Monte-Charlotte
Au-Pied-de-Cochon-Montreal's Menu not on Yelp =(
Rec Made on Seasons-Buffet-Las-Vegas
Rec Made on Trattoria-Reggiano-Las-Vegas
Rec Made on noca-Phoenix
Rec Made on Christophers-and-Crush-Lounge-Phoenix
Rec Made on Georges-Famous-Gyros-Scottsdale
Blue-Fin-Sushi-&-Roll-Las-Vegas's Menu not on Yelp =(
Texas-de-Brazil-Las-Vegas Menu not on Yelp
screlped!
screlped!
Les-Deux-Gamins-Montréal Menu not on Yelp
screlped!
Habaneros-Mexican-Grill-Scottsdale Menu not on Yelp
screlped!
screlped!
screlped!
screlped!
screlped!
screlped!
screlped!
Rec Made on Two-Hippies-Beach-House-Outrageous-Tacos-and-Burritos-Phoenix
Les-Deux-Gamins-Montreal's Menu not on Yelp =(
Rec Made on John-Cut

KeyError: 0

In [166]:
db_dict[700001][0]

[u'Island-Sushi-and-Grill-Las-Vegas',
 u'Over-Easy-Phoenix',
 u'Cartel-Coffee-Lab-Tempe',
 u'Morels-French-Steakhouse-&-Bistro-Las-Vegas',
 u'Kabuki-Japanese-Restaurant-Tempe',
 u'Bad-Daddys-Burger-Bar---Dilworth-Charlotte',
 u'Paradise-Garden-Buffet-Las-Vegas',
 u'Forefathers-Gourmet-Cheesesteaks-&-Fries-Tempe',
 u'San-Felipes-Cantina-Tempe',
 u'Humble-Pie-Scottsdale']

In [174]:
len(menus3)

100

In [134]:
recs_db.keys()

[u'Thai-House-Scottsdale',
 u'Two-Hippies-Magic-Mushroom-Burgers-Phoenix',
 u'Red-Velvet-Cafe-Las-Vegas',
 u'Raising-Canes-Las-Vegas',
 u'Lemongrass-Caf\xe9-Las-Vegas',
 u'Brio-Tuscan-Grille-Las-Vegas',
 u'McKoys-Smokehouse-&-Saloon-Charlotte',
 u'Island-Sushi-and-Grill-Las-Vegas',
 u'Yen-Sushi-&-Sake-Bar-Phoenix',
 u'Geisha-A-Go-Go-Scottsdale',
 u'Maizies-Cafe-&-Bistro-Phoenix',
 u'Bad-Daddys-Burger-Bar---Dilworth-Charlotte',
 u'CUT-Las-Vegas',
 u'In-N-Out-Burger-Las-Vegas',
 u'Tommy-Bahama-Restaurant-&-Bar---Las-Vegas-Las-Vegas',
 u'Panna-Thai-Las-Vegas',
 u'Blue-Martini-Las-Vegas',
 u'YOLOS-Las-Vegas',
 u'Over-Easy-Phoenix',
 u'Pizza-a-Metro-Phoenix',
 u'Tomo-Japanese-Cuisine-Goodyear',
 u'Yard-House-Las-Vegas',
 u'China-Mama-Restaurant-Las-Vegas',
 u'Fresh-Mint-Scottsdale',
 u'Zada-Janes-Corner-Cafe-Charlotte',
 u'Honey-Pig-Las-Vegas',
 u'Cadillac-Ranch-Tempe',
 u'Paradise-Garden-Buffet-Las-Vegas',
 u'Sushi-Loca-Las-Vegas',
 u'Pita-Jungle-Glendale',
 u'Morels-French-Steakhouse-&-Bi

In [214]:
recs_db[u'Blue-Adobe-Grille-Mesa'][0]

,Trendiness,Buzz,Score,Controversy
MenuItem,,,,
Enchanted Garden Enchiladas,7.714286,60,7.250000,5.231233
Triple Stacked Enchiladas,7.714286,60,7.250000,5.231233
Double Stacked Enchiladas,5.500000,83,7.375000,6.824312
Carne Adovada,5.000000,28,8.125000,4.914082
Blue Adobe Burger,4.333333,96,7.078559,6.657367
Chip Chop Salad,3.857143,44,7.653509,3.491273
New Mexican Caesar Salad,3.142857,158,7.925609,3.624994
Blue Seafood Enchiladas,2.952381,73,6.801587,3.822603
"Ocean Front, New Mexico",2.619048,46,7.894571,6.389421


In [183]:
len(recs_db3)

131

In [186]:
count = 0
missings = []
rec_list = []
for key, value in recs_db3.iteritems():
    if isinstance(value, str):
        missings.append(key)
        count += 1
    else:
        rec_list.append(key)
131 - count

67

In [157]:
with open('recs_db1.pkl', 'w') as picklefile:
     pickle.dump(recs_db, picklefile)

In [185]:
with open('recs_db2.pkl', 'w') as picklefile:
     pickle.dump(recs_db3, picklefile)

In [164]:
missings[:11]

[u'Sens-Asian-Tapas-&-Sake-Bar-Phoenix',
 u'LAVO-Italian-Restaurant-&-Lounge-Las-Vegas',
 u'Lemongrass-Caf\xe9-Las-Vegas',
 u'Le-Cartet-Montreal',
 u'Schwartzs-Montreal',
 u'Wienstein-&-Gavinos-Montreal',
 u'Reubens-Deli-&-Steaks-Montreal',
 u'True-Food-Kitchen-Phoenix',
 u'Am\xe9lies-French-Bakery-Charlotte',
 u'Texas-de-Brazil-Las-Vegas',
 u'Forefathers-Gourmet-Cheesesteaks-&-Fries-Tempe']

In [216]:
resto = rec_list[9]

print resto
recs_db3[resto][0]

Krazy-Buffet-Las-Vegas


,Trendiness,Buzz,Score,Controversy
MenuItem,,,,
Sushi,10.000000,56,6.517857,7.842964
Snow Crab Legs,7.941176,41,6.646341,7.647871
Oysters,2.352941,23,6.580882,5.232809
Steamed Seabass,1.470588,6,7.083333,3.562748
Fried Shrimp,1.274510,26,6.332418,6.475509
Coconut Shrimp,1.176471,6,6.666667,6.706349
Chicken Wings,0.588235,9,5.555556,3.353175


In [35]:
print resto
atts[resto]

Cathay-House-Restaurant-Las-Vegas


{'cats': [u'Asian Fusion',
  u'Cantonese',
  u'Dim Sum',
  u'Chinese',
  u'Restaurants'],
 'city': u'Las Vegas',
 'name': u'Cathay House Restaurant',
 'num_reviews': 242,
 'rating': 3.0}

In [205]:
resto_dict = {}
for rec in rec_list:
    resto_dict[rec] = atts[rec]

In [207]:
len(resto_dict)

67

In [208]:
len(atts)

52730

In [209]:
import json

In [210]:
json.dumps(resto_dict, ensure_ascii=False)

u'{"LAVO-Italian-Restaurant-&-Lounge-Las-Vegas": {"city": "Las Vegas", "cats": ["Bars", "Dance Clubs", "Nightlife", "Lounges", "Italian", "Restaurants"], "rating": 3.5, "name": "LAVO Italian Restaurant & Lounge", "num_reviews": 1069}, "Maizies-Cafe-&-Bistro-Phoenix": {"city": "Phoenix", "cats": ["American (New)", "Cafes", "Restaurants"], "rating": 4.0, "name": "Maizie\'s Cafe & Bistro", "num_reviews": 369}, "Tomo-Japanese-Cuisine-Goodyear": {"city": "Goodyear", "cats": ["Asian Fusion", "Sushi Bars", "Japanese", "Restaurants"], "rating": 4.0, "name": "Tomo Japanese Cuisine", "num_reviews": 168}, "District-American-Kitchen-and-Wine-Bar-Phoenix": {"city": "Phoenix", "cats": ["Wine Bars", "Bars", "American (New)", "Nightlife", "Restaurants"], "rating": 3.5, "name": "District American Kitchen and Wine Bar", "num_reviews": 273}, "The-Dhaba-Tempe": {"city": "Tempe", "cats": ["Gluten-Free", "Indian", "Vegan", "Restaurants"], "rating": 4.0, "name": "The Dhaba", "num_reviews": 298}, "Island-Sush

In [211]:
with open('restos.json', 'w') as fp:
    json.dump(resto_dict, fp)

In [85]:
with open('rec_db0.pkl', 'r') as picklefile:
     recs_db = pickle.load(picklefile)

In [98]:
len(recs_db)

215

In [22]:
restos = recs_db.keys()

In [99]:
resto = restos[2]

In [100]:
print resto
recs_db[restos[1]][0]

Cathay-House-Restaurant-Las-Vegas


,Trendiness,Buzz,Score,Controversy
MenuItem,,,,
Bunch O' Chicken,10.000000,31,8.387097,4.587613
Boom Dip,2.619048,7,8.928571,1.285334
Lawrence's Stuffed 'Shrooms,1.904762,7,7.857143,9.640005
Kickbutt Garlic Bread,1.825397,25,7.521368,7.221072


In [32]:
recs_db[restos[1]][0].reset_index().columns

Index([u'MenuItem', u'Trendiness', u'Buzz', u'Score', u'Controversy'], dtype='object')

In [116]:
df_list = []
rest_index = []
rev_index = []
i = 0
for key, value in recs_db.iteritems():
    if isinstance(value, str) or isinstance(value[0],str):
        pass
    else:
        df = value[0].reset_index()
        df['Id'] = np.repeat(key,len(df))
        df['City'] = np.repeat(atts[key]['city'],len(df))
        df['Rating'] = np.repeat(atts[key]['rating'],len(df))
        df['Resto'] = np.repeat(atts[key]['name'],len(df))
        df['RestID'] = np.repeat(i, len(df))
        df_list.append(df)
        rest_index.append((i, key))
        rev_index.append((key,i))
        i += 1

In [195]:
len(df_list)
df_list[0]

,MenuItem,Trendiness,Buzz,Score,Controversy,Id,City,Rating,Resto,RestID
0,Common Cobb,3.601190,86,8.608672,8.284559,The-Common-Market-Charlotte,Charlotte,4.5,The Common Market,0
1,Vegetarian Wrap,2.023810,12,8.541667,5.431357,The-Common-Market-Charlotte,Charlotte,4.5,The Common Market,0
2,Pimento Cheese,1.428571,7,9.285714,1.923077,The-Common-Market-Charlotte,Charlotte,4.5,The Common Market,0


In [103]:
pd.DataFrame(df_list[2].ix[0]).T

,MenuItem,Trendiness,Buzz,Score,Controversy,Id,City,Rating,Resto,RestID
0,Eib Root Bear,1.523482,260,7.302726,2.766804,Ellis-Island-Casino-&-Brewery-Las-Vegas,Las Vegas,4,Ellis Island Casino & Brewery,2


In [106]:
dfm = pd.concat([df_list[0],df_list[1]],axis=0)

In [107]:
for i in range(2, len(df_list)):
    dfm = pd.concat([dfm, df_list[i]], axis=0)

In [108]:
len(dfm)

2053

In [109]:
dfm.tail()

,MenuItem,Trendiness,Buzz,Score,Controversy,Id,City,Rating,Resto,RestID
18,Creamed Spinach,0.384615,12,6.041667,5.313552,Don-&-Charlies-Scottsdale,Scottsdale,4,Don & Charlie's,140
19,Bbq Chicken*,0.307692,5,6.000000,10.000000,Don-&-Charlies-Scottsdale,Scottsdale,4,Don & Charlie's,140
20,Peach Cosmo,0.153846,6,5.416667,1.788721,Don-&-Charlies-Scottsdale,Scottsdale,4,Don & Charlie's,140
21,French Onion Soup,-0.205128,16,4.333333,4.730640,Don-&-Charlies-Scottsdale,Scottsdale,4,Don & Charlie's,140
22,Add Chicken Schnitzel*,-0.307692,5,4.000000,10.000000,Don-&-Charlies-Scottsdale,Scottsdale,4,Don & Charlie's,140


In [77]:
import pymysql
from sqlalchemy import create_engine

In [93]:
dfm.to_csv('recSQLindexed.csv', encoding='utf-8')

In [82]:
pd.read_sql_query('''SELECT * FROM RecsDB0 LIMIT 5''',cnx)

,menuitem,trendiness,buzz,score,controversy,id,city,rating,resto
0,Common Cobb,3.601190,86,8.608672,8.284559,The-Common-Market-Charlotte,Charlotte,4.5,The Common Market
1,Vegetarian Wrap,2.023810,12,8.541667,5.431357,The-Common-Market-Charlotte,Charlotte,4.5,The Common Market
2,Pimento Cheese,1.428571,7,9.285714,1.923077,The-Common-Market-Charlotte,Charlotte,4.5,The Common Market
3,Grilled Fish Tacos,3.581081,41,8.231707,4.041332,Roaring-Fork-Scottsdale,Scottsdale,4.0,Roaring Fork
4,The Roaring Fork Big Ass Burger,2.666667,471,7.962845,3.510540,Roaring-Fork-Scottsdale,Scottsdale,4.0,Roaring Fork


In [84]:
with open('pandas_recs.pkl', 'w') as picklefile:
     pickle.dump(dfm, picklefile)

In [186]:
recs_table = dfm.reset_index().T.to_dict()

{0: {'Buzz': 86.0,
  'City': u'Charlotte',
  'Controversy': 8.284558866503684,
  'Id': u'The-Common-Market-Charlotte',
  'MenuItem': u'Common Cobb',
  'Rating': 4.5,
  'RestID': 0,
  'Resto': u'The Common Market',
  'Score': 8.60867161072079,
  'Trendiness': 3.601190476190476,
  'index': 0},
 1: {'Buzz': 12.0,
  'City': u'Charlotte',
  'Controversy': 5.431356837606836,
  'Id': u'The-Common-Market-Charlotte',
  'MenuItem': u'Vegetarian Wrap',
  'Rating': 4.5,
  'RestID': 0,
  'Resto': u'The Common Market',
  'Score': 8.541666666666668,
  'Trendiness': 2.0238095238095237,
  'index': 1},
 2: {'Buzz': 7.0,
  'City': u'Charlotte',
  'Controversy': 1.9230769230769234,
  'Id': u'The-Common-Market-Charlotte',
  'MenuItem': u'Pimento Cheese',
  'Rating': 4.5,
  'RestID': 0,
  'Resto': u'The Common Market',
  'Score': 9.285714285714286,
  'Trendiness': 1.4285714285714286,
  'index': 2},
 3: {'Buzz': 41.0,
  'City': u'Scottsdale',
  'Controversy': 4.041331505573804,
  'Id': u'Roaring-Fork-Scottsd

In [110]:
with open('recs_table.pkl', 'w') as picklefile:
    pickle.dump(dfm.reset_index().T.to_dict(), picklefile)

In [115]:
dict(rest_index)[0]

u'The-Common-Market-Charlotte'

In [113]:
with open('id_min.pkl', 'r') as picklefile:
     id_min = pickle.load(picklefile)

In [114]:
id_min

,business_id,name,categories,city,stars,review_count,latitude,longitude,CityName
0,vcNAWiLM4dR7D2nwwJ7nCA,"Eric Goldberg, MD","[Doctors, Health & Medical]",Phoenix,3.5,9,33.499313,-111.983758,"Eric-Goldberg,-MD-Phoenix"
1,UsFtqoBl7naz8AVUBZMjQQ,Clancy's Pub,[Nightlife],Dravosburg,3.5,4,40.350519,-79.886930,Clancys-Pub-Dravosburg
2,cE27W9VPgO88Qxe4ol6y_g,Cool Springs Golf Center,"[Active Life, Mini Golf, Golf]",Bethel Park,2.5,5,40.356896,-80.015910,Cool-Springs-Golf-Center-Bethel-Park
3,HZdLhv6COCleJMo7nPl-RA,Verizon Wireless,"[Shopping, Home Services, Internet Service Pro...",Pittsburgh,3.5,3,40.357620,-80.059980,Verizon-Wireless-Pittsburgh
4,mVHrayjG3uZ_RLHkLj-AMg,Emil's Lounge,"[Bars, American (New), Nightlife, Lounges, Res...",Braddock,4.5,11,40.408735,-79.866351,Emils-Lounge-Braddock
5,KayYbHCt-RkbGcPdGOThNg,Alexion's Bar & Grill,"[Bars, American (Traditional), Nightlife, Rest...",Carnegie,4.0,15,40.415517,-80.067534,Alexions-Bar-&-Grill-Carnegie
6,b12U9TFESStdy7CsTtcOeg,Flynn's E W Tire Service Center,"[Auto Repair, Automotive]",Carnegie,1.5,5,40.394588,-80.084454,Flynns-E-W-Tire-Service-Center-Carnegie
7,Sktj1eHQFuVa-M4bgnEh8g,Forsythe Miniature Golf & Snacks,"[Active Life, Mini Golf]",Carnegie,4.0,4,40.405404,-80.076267,Forsythe-Miniature-Golf-&-Snacks-Carnegie
8,3ZVKmuK2l7uXPE6lXY4Dbg,Quaker State Construction,"[Home Services, Contractors]",Carnegie,2.5,3,40.406324,-80.090357,Quaker-State-Construction-Carnegie
9,wJr6kSA5dchdgOdwH6dZ2w,Kings Family Restaurant,"[Burgers, Breakfast & Brunch, American (Tradit...",Carnegie,3.5,8,40.387732,-80.092874,Kings-Family-Restaurant-Carnegie


In [118]:
ind = dict(rev_index)

In [120]:
id_min['RestID'] = id_min['CityName'].apply(lambda x: ind[x] if x in ind else np.nan)

In [121]:
len(id_min.dropna())

170

In [129]:
dfm.groupby('RestID')['Rating']

RestID
0      4.5
1      4.0
2      4.0
3      4.0
4      4.0
5      4.0
6      4.0
7      4.5
8      3.5
9      3.5
10     3.5
11     3.5
12     4.0
13     4.0
14     4.0
15     4.0
16     3.0
17     4.0
18     3.5
19     4.0
20     4.0
21     3.5
22     4.0
23     4.0
24     3.5
25     4.5
26     4.0
27     4.0
28     3.0
29     3.0
      ... 
111    4.0
112    4.0
113    4.0
114    3.5
115    2.5
116    4.0
117    3.0
118    4.0
119    4.0
120    4.0
121    3.5
122    4.5
123    3.5
124    3.0
125    3.5
126    4.0
127    3.0
128    3.5
129    3.5
130    4.0
131    3.5
132    4.0
133    4.5
134    4.0
135    3.5
136    4.5
137    3.5
138    4.0
139    4.5
140    4.0
Name: Rating, dtype: float64

In [131]:
id_min['CityName'].select(crit=(lambda x: x in ind.keys()))

/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if __name__ == '__main__':


Series([], Name: CityName, dtype: object)

In [167]:
rest_info = id_min.dropna().sort(['RestID', 'review_count'], ascending=[True, False])

In [162]:
len(rest_info)

170

In [168]:
rest_info

,business_id,name,categories,city,stars,review_count,latitude,longitude,CityName,RestID
9357,ioXu3MhPt_v8f5wTWJX4ag,The Common Market,"[Arts & Entertainment, Music Venues, Food, Bee...",Charlotte,4.5,159,35.218834,-80.811799,The-Common-Market-Charlotte,0
31011,FMzJJjpd4Pd311StIKkVKA,The Common Market,"[Food, Delis, Farmers Market, Restaurants]",Charlotte,4.5,133,35.215705,-80.856667,The-Common-Market-Charlotte,0
12421,mxrXVZWc6PWk81gvOVNOUw,Roaring Fork,"[Steakhouses, Bars, American (Traditional), Ni...",Scottsdale,4.0,514,33.508695,-111.927080,Roaring-Fork-Scottsdale,1
14541,tqu42L0qXzkvYKSruOz0IA,Ellis Island Casino & Brewery,"[Arts & Entertainment, American (Traditional),...",Las Vegas,4.0,1229,36.113025,-115.163485,Ellis-Island-Casino-&-Brewery-Las-Vegas,2
15943,kFVapB4z_5VoOnYazF_ERA,Oregano's Pizza Bistro,"[Italian, Pizza, Restaurants]",Scottsdale,4.0,317,33.483317,-111.926665,Oreganos-Pizza-Bistro-Scottsdale,3
5415,LMG0zsAkUSscIvmV9vvm3A,Oregano's Pizza Bistro,"[Pizza, Restaurants]",Scottsdale,4.0,263,33.581903,-111.925214,Oreganos-Pizza-Bistro-Scottsdale,3
14192,fkVBloMEFJG2HTgeCxeNDg,Black Bear Diner,"[Breakfast & Brunch, American (Traditional), D...",Las Vegas,4.0,484,36.100346,-115.227476,Black-Bear-Diner-Las-Vegas,4
7103,fjRBF1-0JlJoxH5_jDANgA,Gaetano's Ristorante,"[Restaurants, Italian]",Henderson,4.0,197,36.002856,-115.110654,Gaetanos-Ristorante-Henderson,5
11209,uPDTnuR-sOi47blHH943lg,Roberto's Authentic Mexican Food,"[Mexican, Restaurants]",Anthem,4.0,140,33.844328,-112.133957,Robertos-Authentic-Mexican-Food-Anthem,6
8638,sF8rL9XOz4v48YdTucaBGQ,Piper's Pub,"[Irish, Bars, British, Pubs, Nightlife, Restau...",Pittsburgh,4.5,182,40.428434,-79.979466,Pipers-Pub-Pittsburgh,7


In [146]:
len(rest_info['RestID'].unique())

141

In [154]:
rest_info['resto'] = rest_info['categories'].apply(lambda x: 1 if 'Restaurants' in x else 0)

In [157]:
len(rest_info[rest_info['resto']==1])

168

In [170]:
len(rest_info['RestID'].drop_duplicates())

141

In [174]:
rest_info.drop_duplicates('RestID')

,business_id,name,categories,city,stars,review_count,latitude,longitude,CityName,RestID
9357,ioXu3MhPt_v8f5wTWJX4ag,The Common Market,"[Arts & Entertainment, Music Venues, Food, Bee...",Charlotte,4.5,159,35.218834,-80.811799,The-Common-Market-Charlotte,0
12421,mxrXVZWc6PWk81gvOVNOUw,Roaring Fork,"[Steakhouses, Bars, American (Traditional), Ni...",Scottsdale,4.0,514,33.508695,-111.927080,Roaring-Fork-Scottsdale,1
14541,tqu42L0qXzkvYKSruOz0IA,Ellis Island Casino & Brewery,"[Arts & Entertainment, American (Traditional),...",Las Vegas,4.0,1229,36.113025,-115.163485,Ellis-Island-Casino-&-Brewery-Las-Vegas,2
15943,kFVapB4z_5VoOnYazF_ERA,Oregano's Pizza Bistro,"[Italian, Pizza, Restaurants]",Scottsdale,4.0,317,33.483317,-111.926665,Oreganos-Pizza-Bistro-Scottsdale,3
14192,fkVBloMEFJG2HTgeCxeNDg,Black Bear Diner,"[Breakfast & Brunch, American (Traditional), D...",Las Vegas,4.0,484,36.100346,-115.227476,Black-Bear-Diner-Las-Vegas,4
7103,fjRBF1-0JlJoxH5_jDANgA,Gaetano's Ristorante,"[Restaurants, Italian]",Henderson,4.0,197,36.002856,-115.110654,Gaetanos-Ristorante-Henderson,5
11209,uPDTnuR-sOi47blHH943lg,Roberto's Authentic Mexican Food,"[Mexican, Restaurants]",Anthem,4.0,140,33.844328,-112.133957,Robertos-Authentic-Mexican-Food-Anthem,6
8638,sF8rL9XOz4v48YdTucaBGQ,Piper's Pub,"[Irish, Bars, British, Pubs, Nightlife, Restau...",Pittsburgh,4.5,182,40.428434,-79.979466,Pipers-Pub-Pittsburgh,7
13147,iNT6pcUugoXsDlZgRTyhtw,Caffe Boa,"[Italian, American (New), Breakfast & Brunch, ...",Tempe,3.5,361,33.426651,-111.940428,Caffe-Boa-Tempe,8
6983,PXXFn3kalXTgaWwndlQWHw,Sweet Tomatoes,"[Vegetarian, Buffets, Salad, Restaurants]",Henderson,3.5,156,36.055714,-115.046876,Sweet-Tomatoes-Henderson,9


In [181]:
info = rest_info.drop_duplicates('RestID').drop('business_id',1)

In [225]:
cities = info['city'].unique().tolist()

In [226]:
with open('city_list.pkl', 'w') as picklefile:
    pickle.dump(cities, picklefile)

In [205]:
info['RestID'] = info['RestID'].apply(lambda x: x.astype(int))

In [206]:
master_dict = info.reset_index().T.to_dict()

In [182]:
info.reset_index().T.to_dict()

{0: {'CityName': u'The-Common-Market-Charlotte',
  'RestID': 0.0,
  u'categories': [u'Arts & Entertainment',
   u'Music Venues',
   u'Food',
   u'Beer, Wine & Spirits',
   u'Delicatessen',
   u'Delis',
   u'Nightlife',
   u'Restaurants'],
  u'city': u'Charlotte',
  'index': 9357,
  u'latitude': 35.2188339,
  u'longitude': -80.8117986,
  u'name': u'The Common Market',
  u'review_count': 159,
  u'stars': 4.5},
 1: {'CityName': u'Roaring-Fork-Scottsdale',
  'RestID': 1.0,
  u'categories': [u'Steakhouses',
   u'Bars',
   u'American (Traditional)',
   u'Nightlife',
   u'Restaurants'],
  u'city': u'Scottsdale',
  'index': 12421,
  u'latitude': 33.5086952779617,
  u'longitude': -111.927080154419,
  u'name': u'Roaring Fork',
  u'review_count': 514,
  u'stars': 4.0},
 2: {'CityName': u'Ellis-Island-Casino-&-Brewery-Las-Vegas',
  'RestID': 2.0,
  u'categories': [u'Arts & Entertainment',
   u'American (Traditional)',
   u'Food',
   u'Breweries',
   u'Casinos',
   u'Restaurants'],
  u'city': u'Las

In [185]:
with open('resto_info.pkl', 'w') as picklefile:
    pickle.dump(info.reset_index().T.to_dict(), picklefile)

In [189]:
index_df = pd.merge(left=info, right=dfm, how='left', left_on= 'RestID', right_on='RestID')

In [193]:
index_df.T.to_dict()

{0: {'Buzz': 86.0,
  'City': u'Charlotte',
  'CityName': u'The-Common-Market-Charlotte',
  'Controversy': 8.284558866503684,
  'Id': u'The-Common-Market-Charlotte',
  'MenuItem': u'Common Cobb',
  'Rating': 4.5,
  'RestID': 0.0,
  'Resto': u'The Common Market',
  'Score': 8.60867161072079,
  'Trendiness': 3.601190476190476,
  u'categories': [u'Arts & Entertainment',
   u'Music Venues',
   u'Food',
   u'Beer, Wine & Spirits',
   u'Delicatessen',
   u'Delis',
   u'Nightlife',
   u'Restaurants'],
  u'city': u'Charlotte',
  u'latitude': 35.2188339,
  u'longitude': -80.8117986,
  u'name': u'The Common Market',
  u'review_count': 159,
  u'stars': 4.5},
 1: {'Buzz': 12.0,
  'City': u'Charlotte',
  'CityName': u'The-Common-Market-Charlotte',
  'Controversy': 5.431356837606836,
  'Id': u'The-Common-Market-Charlotte',
  'MenuItem': u'Vegetarian Wrap',
  'Rating': 4.5,
  'RestID': 0.0,
  'Resto': u'The Common Market',
  'Score': 8.541666666666668,
  'Trendiness': 2.0238095238095237,
  u'categorie

In [191]:
len(index_df)

2053

In [218]:
with open('master_dict.pkl', 'w') as picklefile:
    pickle.dump(master_dict, picklefile)

In [197]:
df_list[0].T.to_dict()

{0: {'Buzz': 86.0,
  'City': u'Charlotte',
  'Controversy': 8.284558866503684,
  'Id': u'The-Common-Market-Charlotte',
  'MenuItem': u'Common Cobb',
  'Rating': 4.5,
  'RestID': 0,
  'Resto': u'The Common Market',
  'Score': 8.60867161072079,
  'Trendiness': 3.601190476190476},
 1: {'Buzz': 12.0,
  'City': u'Charlotte',
  'Controversy': 5.431356837606836,
  'Id': u'The-Common-Market-Charlotte',
  'MenuItem': u'Vegetarian Wrap',
  'Rating': 4.5,
  'RestID': 0,
  'Resto': u'The Common Market',
  'Score': 8.541666666666668,
  'Trendiness': 2.0238095238095237},
 2: {'Buzz': 7.0,
  'City': u'Charlotte',
  'Controversy': 1.9230769230769234,
  'Id': u'The-Common-Market-Charlotte',
  'MenuItem': u'Pimento Cheese',
  'Rating': 4.5,
  'RestID': 0,
  'Resto': u'The Common Market',
  'Score': 9.285714285714286,
  'Trendiness': 1.4285714285714286}}

In [212]:
for key, value in master_dict.iteritems():
    print key
    master_dict[key]['menu'] = df_list[key].T.to_dict()
    for k, v in master_dict[key]['menu'].iteritems():
        v['RestID'] = key

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140


In [222]:
master_dict[140]

{'CityName': u'Don-&-Charlies-Scottsdale',
 'RestID': 140,
 u'categories': [u'Steakhouses', u'Barbeque', u'Restaurants'],
 u'city': u'Scottsdale',
 'index': 12500,
 u'latitude': 33.501821,
 u'longitude': -111.919938,
 'menu': {0: {'Buzz': 56,
   'City': u'Scottsdale',
   'Controversy': 6.145640074211503,
   'Id': u'Don-&-Charlies-Scottsdale',
   'MenuItem': u'Add Half a Chicken*',
   'Rating': 4.0,
   'RestID': 140,
   'Resto': u"Don & Charlie's",
   'Score': 6.607142857142857,
   'Trendiness': 5.538461538461538},
  1: {'Buzz': 74,
   'City': u'Scottsdale',
   'Controversy': 3.6927345109250083,
   'Id': u'Don-&-Charlies-Scottsdale',
   'MenuItem': u'Prime Ny Steak',
   'Rating': 4.0,
   'RestID': 140,
   'Resto': u"Don & Charlie's",
   'Score': 6.748400852878465,
   'Trendiness': 5.230769230769231},
  2: {'Buzz': 19,
   'City': u'Scottsdale',
   'Controversy': 1.9306639805254764,
   'Id': u'Don-&-Charlies-Scottsdale',
   'MenuItem': u'Spicy Broasted Chicken',
   'Rating': 4.0,
   'Rest